<a href="https://colab.research.google.com/github/moocdev/12306/blob/master/LangChain_%E4%B8%AD%E6%96%87%E5%85%A5%E9%97%A8%E6%95%99%E7%A8%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

文章已开源，欢迎star
https://github.com/liaokongVFX/LangChain-Chinese-Getting-Started-Guide

#装包以及初始化

In [1]:
!pip install langchain
!pip install openai
!pip install google-search-results
!pip install unstructured
!pip install chromadb
!pip install pinecone-client
!pip install youtube-transcript-api
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''

#使用 LangChain 完成一次问答

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003",max_tokens=1024)
llm("怎么评价人工智能")

'\n\n人工智能是一个极具潜力的领域，它在推动着未来科技发展和社会进步。人工智能已经取得了一定的成就，但仍有很多需要改进的地方，其中包括提高技术的精确度和可靠性，提升人工智能的难度和复杂度，以及更好地理解人类的行为和心理。这些都将需要更多的研究和实践，才能实现人工智能的最终目标。'

#通过 Google 搜索并返回答案

In [ ]:
import os
os.environ["SERPAPI_API_KEY"] = ''

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.agents import AgentType

# 加载 OpenAI 模型
llm = OpenAI(temperature=0,max_tokens=2048)

 # 加载 serpapi 工具
tools = load_tools(["serpapi"])
# 工具加载后都需要初始化，verbose 参数为 True，会打印全部的执行详情
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 运行 agent
agent.run("What's the date today? What great events have taken place today in history?")



> Entering new AgentExecutor chain...
 I should look up the current date and then search for historical events that have taken place on that day.
Action: Search
Action Input: Current date
Observation: Today's current date and time with time zone and date picker: Select locale. en-US. Monday, April 3, 2023. 11:00:05 PM. PST8PDT (GMT-7).
Thought: I should now search for historical events that have taken place on this day.
Action: Search
Action Input: Historical events April 3
Observation: No good search result found
Thought: I should narrow down my search to a specific year.
Action: Search
Action Input: Historical events April 3 2023
Observation: No good search result found
Thought: I should try searching for a specific event.
Action: Search
Action Input: Apollo 16 launch April 3 1972
Observation: The fifth Moon landing mission began with the April 16, 1972 launch of Apollo 16. The giant Saturn V rocket lifted off from Launch Pad 39A at NASA's Kennedy Space Center (KSC) in Florida with

'Today is Monday, April 3, 2023. On this day in 1972, the Apollo 16 mission was launched, the fifth Moon landing mission.'

#对超长文本进行总结

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI

# 导入文本
loader = UnstructuredFileLoader("/content/sample_data/data/lg_test.txt")
# 将文本转成 Document 对象
document = loader.load()
print(f'documents:{len(document)}')

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

# 切分文本
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

# 加载 llm 模型
llm = OpenAI(max_tokens=1500)

# 创建总结链
chain = load_summarize_chain(llm, chain_type="refine", verbose=True)

# 执行总结链，（为了快速演示，只总结前5段）
chain.run(split_documents[:5])

documents:1
documents:317


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"声明：本书为爱奇电子书(www.i7wu.cn)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何关系。

--------------------------用户上传之内容开始--------------------------------

《地藏心经》

作者：铸剑师无名

正文

第一第十五章 天下势，渡江（一）

“渝州陆家？！”

虽然原本的那个秦逸，每日只知道苦读诗书，从未与商贾们打过交道，但是渝州陆家的名声，他还是知道。

陆家三代为官，官至两江总督，五代经商，百年经营，家私何止千万，直至今朝，俨然已是江南一等士族大户。渝州陆氏以皮货起家，乃是西北之地数得上号的商户，西北之地所产的皮货，有三成经他们之手卖往江南。

若只是如此，陆氏也不过是一头肥硕的羔羊，只待他人宰杀。

陆氏三代家主都极具雄韬伟略，以千金买官，以万金开路，更是在蛮夷南侵之时，倾尽家资招兵买马，拒十万蛮夷铁骑于侯关外，短短三年间，便一手扶持起了都护大将军——苏和，抗夷大将军——邓昌。"


CONCISE SUMMARY:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary
We have provided an existing summary up to a certain point: 
《地藏心经》是由铸剑师无名创作的作品，本书讲述了渝州陆家三代家主的传奇故事，他们以皮货起家，在蛮夷南侵时招兵买马，一手扶持起了都护大将军和抗夷大将军，让渝州陆家走向了巅峰。
We have the opportunity to refine the existing summary(only if n

'\n《地藏心经》讲述了渝州陆家三代家主的传奇故事，他们以皮货起家，招兵买马，扶持起都护大将军和抗夷大将军，握住兵权，一路仕途平坦，百年来，人才辈出，雄踞渝、豫两地，掌握江南数万水军，走向巅峰，尊贵至极，不可小觑的贵族。秦家在西北之地声名远播，善名百里，五代行善，在中原数千年来，独此一家，就连蛮夷赫连氏族入侵中原时，都刻意避开了晋中秦家，屠戮真正善人，会被狼神抛弃。陆家车行也声名远扬，由管事宋老虎管理，秦逸受其邀请，一同登船渡江，令渝州陆家走向了巅峰，并受到众多贵族的尊敬。'

#构建本地知识库问答机器人

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA

# 加载文件夹中的所有txt类型的文件
loader = DirectoryLoader('/content/sample_data/data/', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings()
# 将 document 通过 openai 的 embeddings 对象计算 embedding向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(split_docs, embeddings)

# 创建问答对象
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch,return_source_documents=True)
# 进行问答
result = qa({"query": "科大讯飞今年第一季度收入是多少？"})
print(result)

{'query': '科大讯飞今年第一季度收入是多少？', 'result': ' 根据报告，科大讯飞今年第一季度的收入为126.5亿人民币。', 'source_documents': [Document(page_content='虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显慢了下来，这其中最重要的一个因素，便是由于第一个财季受到了病毒的冲击，导致了国内不少工程进度推迟。在讯飞的智能教育商业计划中，第三季度有10个左右的延期，合约金额接近16个。另外，安徽省方面，为强化对省内规划工程的协调，在第三季度内，大部分大型工程的投标都已经停止，这也是造成今年第三季度，科大讯飞收入的增幅低于预计的主要因素。', metadata={'source': '/content/sample_data/data/t1.txt'}), Document(page_content='科大讯飞前三季营收曝光，用AI赋能企业转型推出了多款SaaS产品', metadata={'source': '/content/sample_data/data/t1.txt'}), Document(page_content='而在十月二十八日晚上，在一份关于科技的报告中，复旦讯飞公布了2022年第三季度的财报。据财报，今年第一季度，科技讯飞的收入为126.5亿人民币，较上年同期增加16.5%。该公司在应对新冠肺炎疫情的过程中，在其核心产业领域，持续取得了较好的发展。', metadata={'source': '/content/sample_data/data/t1.txt'}), Document(page_content='据该份财报，截至今年第三季度，科技讯飞的营运资金净流入率较去年增加百分之三十二，虽然受到了严峻的经济形势影响，但科技讯飞的营运资金和营业收入仍维持着强劲的发展势头。', metadata={'source': '/content/sample_data/data/t1.txt'})]}


# 构建向量索引数据库

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

import pinecone

# 初始化 pinecone
pinecone.init(
  api_key="你的api key",
  environment="你的Environment"
)

loader = DirectoryLoader('/content/sample_data/data/', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings()

index_name="liaokong-test"
# 持久化数据
# docsearch = Pinecone.from_texts([t.page_content for t in split_docs], embeddings, index_name=index_name)

# 加载数据
docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = "科大讯飞今年第一季度收入是多少？"
docs = docsearch.similarity_search(query, include_metadata=True)

llm = OpenAI(temperature=0)
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)
chain.run(input_documents=docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

科大讯飞前三季营收曝光，用AI赋能企业转型推出了多款SaaS产品

而在十月二十八日晚上，在一份关于科技的报告中，复旦讯飞公布了2022年第三季度的财报。据财报，今年第一季度，科技讯飞的收入为126.5亿人民币，较上年同期增加16.5%。该公司在应对新冠肺炎疫情的过程中，在其核心产业领域，持续取得了较好的发展。

据该份财报，截至今年第三季度，科技讯飞的营运资金净流入率较去年增加百分之三十二，虽然受到了严峻的经济形势影响，但科技讯飞的营运资金和营业收入仍维持着强劲的发展势头。

虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显慢了下来，这其中最重要的一个因素，便是由于第一个财季受到了病毒的冲击，导致了国内不少工程进度推迟。在讯飞的智能教育商业计划中，第三季度有10个左右的延期，合约金额接近16个。另外，安徽省方面，为强化对省内规划工程的协调，在第三季度内，大部分大型工程的投标都已经停止，这也是造成今年第三季度，科大讯飞收入的增幅低于预计的主要因素。

虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显慢了下来，这其中最重要的一个因素，便是由于第一个财季受到了病毒的冲击，导致了国内不少工程进度推迟。在讯飞的智能教育商业计划中，第三季度有10个左右的延期，合约金额接近16个。另外，安徽省方面，为强化对省内规划工程的协调，在第三季度内，大部分大型工程的投标都已经停止，这也是造成今年第三季度，科大讯飞收入的增幅低于预计的主要因素。

虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显

' 根据财报，今年第一季度，科技讯飞的收入为126.5亿人民币。'

#使用GPT3.5模型构建油管频道问答机器人


In [ ]:
import os

from langchain.document_loaders import YoutubeLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ChatVectorDBChain, ConversationalRetrievalChain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)

# 加载 youtube 频道
loader = YoutubeLoader.from_youtube_channel('https://www.youtube.com/watch?v=Dj60HHy-Kqk')
# 将数据转成 document
documents = loader.load()

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=20
)

# 分割 youtube documents
documents = text_splitter.split_documents(documents)

# 初始化 openai embeddings
embeddings = OpenAIEmbeddings()

# 将数据存入向量存储
vector_store = Chroma.from_documents(documents, embeddings)
# 通过向量存储初始化检索器
retriever = vector_store.as_retriever()

system_template = """
Use the following context to answer the user's question.
If you don't know the answer, say you don't, don't try to make it up. And answer in Chinese.
-----------
{context}
-----------
{chat_history}
"""

# 构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('{question}')
]

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_messages(messages)


# 初始化问答链
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0.1,max_tokens=2048),retriever,qa_prompt=prompt)


chat_history = []
while True:
  question = input('问题：')
  # 开始发送问题 chat_history 为必须参数,用于存储对话历史
  result = qa({'question': question, 'chat_history': chat_history})
  chat_history.append((question, result['answer']))
  print(result['answer'])

问题：这个视频讲了什么？
这个视频是关于 Unreal Engine 5.2 的新实验功能的展示。在视频中介绍了一些新特性，如植被渲染、物理模拟、流体模拟、材质框架等。同时，还展示了一些环境场景的探索和汽车驾驶的游戏体验。视频还介绍了一些可用于生成可编程内容的实验工具。
问题：对环境场景的探索有哪些？
视频中展示了一个可编程的环境场景，包括树木、岩石、雾、虫子、鸟类等，同时还有一个四平方公里的区域可以探索，漫游和越野驾驶。视频还展示了汽车驾驶的游戏体验和实验工具的使用。


KeyboardInterrupt: ignored

#用 OpenAI 连接万种工具

In [ ]:
import os
os.environ["ZAPIER_NLA_API_KEY"] = ''

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper


llm = OpenAI(temperature=.3)

zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(toolkit.get_tools(), llm, agent="zero-shot-react-description", verbose=True)

# 我们可以通过打印的方式看到我们都在 Zapier 里面配置了哪些可以用的工具
for tool in toolkit.get_tools():
  print (tool.name)
  print (tool.description)
  print ("\n\n")

Gmail: Find Email
A wrapper around Zapier NLA actions. The input to this tool is a natural language instruction, for example "get the latest email from my bank" or "send a slack message to the #general channel". Each tool will have params associated with it that are specified as a list. You MUST take into account the params when creating the instruction. For example, if the params are ['Message_Text', 'Channel'], your instruction should be something like 'send a slack message to the #general channel with the text hello world'. Another example: if the params are ['Calendar', 'Search_Term'], your instruction should be something like 'find the meeting in my personal calendar at 3pm'. Do not make up params, they will be explicitly specified in the tool description. If you do not have enough information to fill in the params, just say 'not enough information provided in the instruction, missing <param>'. If you get a none or null response, STOP EXECUTION, do not try to another tool!This too

In [ ]:
agent.run('请用中文总结最后一封"*********@qq.com"发给我的邮件。并将总结发送给"*********@qq.com"')

# 一些有意思的小Tip

##执行多个chain

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# location 链
llm = OpenAI(temperature=1)
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)
location_chain = LLMChain(llm=llm, prompt=prompt_template)

# meal 链
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

# 通过 SimpleSequentialChain 串联起来，第一个答案会被替换第二个中的user_meal，然后再进行询问
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)
review = overall_chain.run("Rome")



> Entering new SimpleSequentialChain chain...
Pasta alla Carbonara.

Ingredients:
- 2 cloves of garlic, minced 
- 2 tablespoons of olive oil 
- 2 cups of cooked spaghetti pasta 
- 4 slices of thick-cut bacon, cooked and chopped 
- 2 eggs, lightly beaten 
- 1/2 cup of Parmesan cheese, grated 
- Salt and black pepper to taste 

Directions:
1. In a large skillet, heat the olive oil over medium heat and add the garlic, stirring until it is fragrant (about 2 minutes).
2. Push the garlic to the side of the pan and add the cooked spaghetti pasta, stirring it around until all of the strands are coated with garlic and oil.
3. Turn off the heat, add the cooked bacon and Parmesan cheese, and stir until combined.
4. Add the beaten eggs and mix until evenly combined.
5. Season to taste with salt and pepper, and serve warm. Enjoy!

> Finished chain.


##结构化输出

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

# 告诉他我们生成的内容需要哪些字段，每个字段类型式啥
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# 初始化解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 生成的格式提示符
# {
#	"bad_string": string  // This a poorly formatted user input string
#	"good_string": string  // This is your response, a reformatted response
#}
format_instructions = output_parser.get_format_instructions()

template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# 讲我们的格式描述嵌入到 prompt 中去，告诉 llm 我们需要他输出什么样格式的内容
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")
llm_output = llm(promptValue)

# 使用解析器进行解析生成的内容
output_parser.parse(llm_output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

##爬取网页并输出JSON数据

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = """在 >>> 和 <<< 之间是网页的返回的HTML内容。
网页是新浪财经A股上市公司的公司简介。
请抽取参数请求的信息。

>>> {requests_result} <<<
请使用如下的JSON格式返回数据
{{
  "company_name":"a",
  "company_english_name":"b",
  "issue_price":"c",
  "date_of_establishment":"d",
  "registered_capital":"e",
  "office_address":"f",
  "Company_profile":"g"

}}
Extracted:"""

prompt = PromptTemplate(
    input_variables=["requests_result"],
    template=template
)

chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt))
inputs = {
  "url": "https://vip.stock.finance.sina.com.cn/corp/go.php/vCI_CorpInfo/stockid/600519.phtml"
}

response = chain(inputs)
print(response['output'])

{
  "company_name":"贵州茅台酒股份有限公司",
  "company_english_name":"Kweichow Moutai Co.,Ltd.",
  "issue_price":"31.39",
  "date_of_establishment":"1999-11-20",
  "registered_capital":"125620万元(CNY)",
  "office_address":"贵州省仁怀市茅台镇",
  "Company_profile":"公司是根据贵州省人民政府黔府函〔1999〕291号文,由中国贵州茅台酒厂有限责任公司作为主发起人,联合贵州茅台酒厂技术开发公司、贵州省轻纺集体工业联社、深圳清华大学研究院、中国食品发酵工业研究院、北京市糖业烟酒公司、江苏省糖烟酒总公司、上海捷强烟草糖酒(集团)有限公司于1999年11月20日共同发起设立的股份有限公司。经中国证监会证监发行字[2001]41号文核准并按照财政部企[2001]56号文件的批复,公司于2001年7月31日在上海证券交易所公开发行7,150万(其中,国有股存量发行650万股)A股股票。主营业务：贵州茅台酒系列产品的生产与销售,饮料、食品、包装材料的生产与销售,防伪技术开发;信息产业相关产品的研制和开发等。"
}


##自定义工具

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = OpenAI(temperature=0)

# 初始化搜索链和计算链
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

# 生成一个功能列表，指明这个 agent 里面都有哪些可用工具
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

# 初始化 agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 执行 agent
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. The Titanic actor hasn't been in a relationship with a woman over the age of ...
Thought: I need to find out Camila Morrone's age
Action: Search
Action Input: "Camila Morrone age"
Observation: 25 years
Thought: I now know the age and can calculate the power
Action: Calculator
Action Input: 25^0.43

> Entering new LLMMathChain chain...
25^0.43
```python
import math
print(math.pow(25, 0.43))
```

Answer: 3.991298452658078

> Finished chain.

Observation: Answer: 3.991298452658078

Thought: I now know the final answer
Final Answer: Camila Morrone's age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


"Camila Morrone's age raised to the 0.43 power is 3.991298452658078."

##使用Memory实现一个带记忆的对话机器人

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

# 初始化 MessageHistory 对象
history = ChatMessageHistory()

# 给 MessageHistory 对象添加对话内容
history.add_ai_message("你好！")
history.add_user_message("中国的首都是哪里？")

# 执行对话
ai_response = chat(history.messages)
print(ai_response)

content='中国的首都是北京。' additional_kwargs={}
